# Criando features

- Esse notebook é a continuação do [1. EDA](1.%20EDA.ipynb)

# Ìndice
1. [Resumo e objetivo](#Resumo-e-Objetivo)
- [Features Básicas](#Features-Básicas)
    - [Feature shop_name](#Features-shop_name)
    - [Feature category_id](#Features-category_id)
    - [Retirando item_name e date](#Retirando-item_name-e-date)
- [Criando Features Novas](#Criando-Features-Novas)
    - [Revenue](#Revenue)
    - [Semestre e ano](#Semestre-e-ano)  
    - [Médias mensais](#Médias-mensais)
        - [Média por loja](#Média-por-loja)
        -  [Média por item](#Média-por-item)
        -  [Média dos itens por loja](#Média-dos-itens-por-loja)
    - [Lagged Variables](#Lagged-Variables) (Em construção)
    - [Latitude e Longitude](#Latitude-e-Longitude)
    - [Unindo os dataframes](#unindo)
- [Apêndice](#Apêndice)
- [Referências](#Referências)

### Coisas a serem feitas:
- [ ] **Comentar o código e deixar as variáveis mais fáceis de serem lidas.**
- Colocar as outras featues.
    - [ ] Lagged. [1](https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/) [2](https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/)
    - [x] Meses.  
    - [ ] Médias.
        - [ ] Fazer a média do item_cnt por loja por item.
        - [ ] Fazer a média do item_cnt por categória.
- [ ] Tem que melhorar muito a explicação das laggeds.
- [ ] Unir os dataframes.

# Carregando os arquivos

In [18]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

path = 'data'

train = pd.read_csv(path + '/sales_train.csv.gz')
test = pd.read_csv(path + '/test.csv.gz').set_index('ID')
items = pd.read_csv(path + '/items.csv')
items_cat = pd.read_csv(path + '/item_categories.csv')
shops = pd.read_csv(path + '/shops.csv')

# Resumo e objetivo

Neste notebook nos vamos criar várias features novas e depois vamos voltar em [1. EDA](1.%20EDA.ipynb) e tentar extrair alguma informação.

# Features Básicas

- [Feature shop_name](#Features-shop_name)
- [Feature category_id](#Features-category_id)
- [Retirando item_name e date](#Retirando-item_name-e-date)

### Features shop_name

A feature categórica shop_name, também possui informação sobre localização e tipos de lojas.

- Адыгея ТЦ "Мега"         => Adygea shopping center "Mega"	
- Волжский ТЦ "Волга Молл" => Volzhsky shopping center "Volga Mall"
- Вологда ТРЦ "Мармелад"   => Shopping center Vologda "Marmalade"	
- Якутск ТЦ "Центральный"  => Centro comercial Yakutsk "Central"	
- Ярославль ТЦ "Альтаир"   => Centro comercial Yaroslavl "Altair"

In [19]:
# Corrigindo uma entrada.
shops.loc[shops.shop_name == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'

# Separando as palavras (split) e escolhendo a primeira (map(lambda x: x[0])).
shops['city'] = shops['shop_name'].str.split(' ').map(lambda x: x[0])

#Corringindo um nome.
shops.loc[shops.city == '!Якутск', 'city'] = 'Якутск'

print('Nome das cidades:\n',shops['city'].head())

Nome das cidades:
 0      Якутск
1      Якутск
2      Адыгея
3    Балашиха
4    Волжский
Name: city, dtype: object


> ### Obs.:
> Note que é possível retirar mais informações dos nomes, como o tipo de estabelecimento (Shopping, Centro comercial, e etc), o tipo do shopping (Volga Mall) e até mesmo a localização na cidade (Central).

Apesar de termos as localizações separadas, a maioria dos algoritmos só enchergam números, sendo o XGboost uma excessão. Então, temos que rotular cada nome de cidade com um número.

<a name="back1"></a> Deixei um exemplo simples [aqui](#go1) de como fazer isso para quem ainda está começando.

In [20]:
shops['city_code'] = LabelEncoder().fit_transform(shops['city'])
shops.head()

,shop_name,shop_id,city,city_code
0,"!Якутск Орджоникидзе, 56 фран",0,Якутск,29
1,"!Якутск ТЦ ""Центральный"" фран",1,Якутск,29
2,"Адыгея ТЦ ""Мега""",2,Адыгея,0
3,"Балашиха ТРК ""Октябрь-Киномир""",3,Балашиха,1
4,"Волжский ТЦ ""Волга Молл""",4,Волжский,2


In [21]:
shops.drop(['shop_name', 'city'], axis='columns', inplace=True)
shops.head()

,shop_id,city_code
0,0,29
1,1,29
2,2,0
3,3,1
4,4,2


### Feature category_id

O mesmo procedimento vai ser realizado para os nomes das categorias. Vamos dividi-las em duas features, tipo e subtipo. Exemplo:
    - Аксессуары - PS2 => Tipo: Acessórios / Subtipo: PS2.
    
Da mesma forma iremos definir um número para cada um dos tipos e subtipos.

In [22]:
# Separando as duas categorias.
items_cat['split'] = items_cat['item_category_name'].str.split('-')

# Criando uma coluna só com o tipo e depois criando os rótulos.
items_cat['type'] = items_cat['split'].map(lambda x: x[0].strip())
items_cat['type_code'] = LabelEncoder().fit_transform(items_cat['type'])

# Criando uma coluna só com o subtipo e depois criando os rótulos.
items_cat['subtype'] = items_cat['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
items_cat['subtype_code'] = LabelEncoder().fit_transform(items_cat['subtype'])

items_cat.head()

,item_category_name,item_category_id,split,type,type_code,subtype,subtype_code
0,PC - Гарнитуры/Наушники,0,"[PC , Гарнитуры/Наушники]",PC,0,Гарнитуры/Наушники,29
1,Аксессуары - PS2,1,"[Аксессуары , PS2]",Аксессуары,1,PS2,9
2,Аксессуары - PS3,2,"[Аксессуары , PS3]",Аксессуары,1,PS3,10
3,Аксессуары - PS4,3,"[Аксессуары , PS4]",Аксессуары,1,PS4,11
4,Аксессуары - PSP,4,"[Аксессуары , PSP]",Аксессуары,1,PSP,13


In [23]:
# Retirando colunas que não tem mais utilidade.
items_cat.drop(['item_category_name', 'split', 'type', 'subtype'], axis='columns', inplace=True)
items_cat.head()

,item_category_id,type_code,subtype_code
0,0,0,29
1,1,1,9
2,2,1,10
3,3,1,11
4,4,1,13


### Retirando item_name e date

 Vou retirar duas colunas do banco de dado, item_name e date. As datas não vão ser precisas porque vou utilizar somente os meses e para isso já temos a coluna date_block_num. No nome dos itens deve ter alguma informação interessante, mas vai ser muito complicado separar essas coisas.

In [24]:
items.drop(['item_name'], axis=1, inplace=True)

In [25]:
train.drop(['date'], axis='columns', inplace=True)

# Criando Features Novas

Criar features é tão importante quanto fazer uma boa EDA. Algumas features podem fazer a diferença para modelo preditivos. Vou abordar a construção das seguintes features:

   - [Revenue](#Revenue)
   - [Semestre e ano](#Semestre-e-ano)  
   - [Médias mensais](#Médias-mensais)
       - [Média por loja](#Média-por-loja)
       -  [Média por item](#Média-por-item)
       -  [Média dos itens por loja](#Média-dos-itens-por-loja)
   - [Lagged Variables](#Lagged-Variables) 
   
 > **Obs.:**
     - Vou criar primeiro features que são combinações de outras do dataset.
     - Depois vamos criar médias e somas com base em meses.
     - Por último entra as variáveis lagged, elas entram por último porque é importante manter a ordem delas.

### Revenue

A feature mais básica de todas é a receita (Dinheiro total ganho por item vendido).

In [26]:
train['revenue'] = train['item_price'] * train['item_cnt_day']

### Somas mensais

In [27]:
train.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue
0,0,59,22154,999.00,1.0,999.00
1,0,25,2552,899.00,1.0,899.00
2,0,25,2552,899.00,-1.0,-899.00
3,0,25,2554,1709.05,1.0,1709.05
4,0,25,2555,1099.00,1.0,1099.00


In [28]:
# Vamos usar muito essas colunas.
group_basic = ['date_block_num', 'shop_id', 'item_id']

# Não faz sentido somar o preço unitário de cada item.
df_train_month = train.drop(['item_price'], axis=1).groupby(group_basic).sum()
df_train_month.rename(columns={'item_cnt_day':'item_cnt_month', 'revenue':'revenue_month'}, inplace=True)
df_train_month.reset_index(inplace=True)
df_train_month.head()

,date_block_num,shop_id,item_id,item_cnt_month,revenue_month
0,0,0,32,6.0,1326.0
1,0,0,33,3.0,1041.0
2,0,0,35,1.0,247.0
3,0,0,43,1.0,221.0
4,0,0,51,2.0,257.0


### Médias mensais

Queremos tentar prever as vendas mensais do itens, então vou criar algumas features com base em médias mensais.

> **Obs.:**
   - Tirar médias do item_cnt_day não é muito preciso, porque algumas contém valores negativos. Mas deve ser pelo menos uma aproximação boa se não tivermos muitos números ímpares e valores pequenos.

### Média por loja

Vamos calcular a média do número de itens vendidos e da receita de cada loja por mês.

In [29]:
train.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue
0,0,59,22154,999.00,1.0,999.00
1,0,25,2552,899.00,1.0,899.00
2,0,25,2552,899.00,-1.0,-899.00
3,0,25,2554,1709.05,1.0,1709.05
4,0,25,2555,1099.00,1.0,1099.00


In [30]:
aux = train.drop(['item_id'], axis=1)
aux = aux.groupby(['date_block_num', 'shop_id']).mean()
aux.rename(columns={'item_cnt_day':'mean_item_cnt_per_shop', 'revenue':'mean_revenue_per_shop',
                   'item_price':'mean_item_price_per_shop'},
          inplace=True)

df_train_month = pd.merge(df_train_month, aux, on=['date_block_num', 'shop_id'], how='left')
df_train_month.head()

,date_block_num,shop_id,item_id,item_cnt_month,revenue_month,mean_item_price_per_shop,mean_item_cnt_per_shop,mean_revenue_per_shop
0,0,0,32,6.0,1326.0,531.262049,1.163781,618.90507
1,0,0,33,3.0,1041.0,531.262049,1.163781,618.90507
2,0,0,35,1.0,247.0,531.262049,1.163781,618.90507
3,0,0,43,1.0,221.0,531.262049,1.163781,618.90507
4,0,0,51,2.0,257.0,531.262049,1.163781,618.90507


### Média por item

Vou calcular a média do número de itens vendidos e receita por item em cada mês.

In [31]:
aux = train.drop(['shop_id'], axis=1)
aux = aux.groupby(['date_block_num', 'item_id']).mean()
aux.rename(columns={'item_cnt_day':'mean_item_cnt_per_item', 'revenue':'mean_revenue_per_item',
                   'item_price':'mean_item_price_per_item'},
          inplace=True)
df_train_month = pd.merge(df_train_month, aux, on=['date_block_num', 'item_id'], how='left')
df_train_month.head()

,date_block_num,shop_id,item_id,item_cnt_month,revenue_month,mean_item_price_per_shop,mean_item_cnt_per_shop,mean_revenue_per_shop,mean_item_price_per_item,mean_item_cnt_per_item,mean_revenue_per_item
0,0,0,32,6.0,1326.0,531.262049,1.163781,618.90507,338.110349,1.328889,451.181600
1,0,0,33,3.0,1041.0,531.262049,1.163781,618.90507,488.517241,1.051724,514.327586
2,0,0,35,1.0,247.0,531.262049,1.163781,618.90507,390.709091,1.418182,549.272727
3,0,0,43,1.0,221.0,531.262049,1.163781,618.90507,234.500000,1.000000,234.500000
4,0,0,51,2.0,257.0,531.262049,1.163781,618.90507,230.461538,1.000000,230.461538


### Média por categoria

Vai ser preciso unir as partes do dataset

In [32]:
train = pd.merge(train, items, on=['item_id'], how='left')
train = pd.merge(train, items_cat, on=['item_category_id'], how='left')
train.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue,item_category_id,type_code,subtype_code
0,0,59,22154,999.00,1.0,999.00,37,11,1
1,0,25,2552,899.00,1.0,899.00,58,13,27
2,0,25,2552,899.00,-1.0,-899.00,58,13,27
3,0,25,2554,1709.05,1.0,1709.05,58,13,27
4,0,25,2555,1099.00,1.0,1099.00,56,13,3


In [36]:
df_train_month = pd.merge(df_train_month, items, on=['item_id'], how='left')
df_train_month = pd.merge(df_train_month, items_cat, on=['item_category_id'], how='left')
df_train_month.head()

,date_block_num,shop_id,item_id,item_cnt_month,revenue_month,mean_item_price_per_shop,mean_item_cnt_per_shop,mean_revenue_per_shop,mean_item_price_per_item,mean_item_cnt_per_item,mean_revenue_per_item,item_category_id,type_code,subtype_code
0,0,0,32,6.0,1326.0,531.262049,1.163781,618.90507,338.110349,1.328889,451.181600,40,11,4
1,0,0,33,3.0,1041.0,531.262049,1.163781,618.90507,488.517241,1.051724,514.327586,37,11,1
2,0,0,35,1.0,247.0,531.262049,1.163781,618.90507,390.709091,1.418182,549.272727,40,11,4
3,0,0,43,1.0,221.0,531.262049,1.163781,618.90507,234.500000,1.000000,234.500000,40,11,4
4,0,0,51,2.0,257.0,531.262049,1.163781,618.90507,230.461538,1.000000,230.461538,57,13,8


In [38]:
aux = train.drop(['shop_id', 'item_id', 'type_code', 'subtype_code'], axis=1)
aux = aux.groupby(['date_block_num', 'item_category_id']).mean()
aux.rename(columns={'item_cnt_day':'mean_item_cnt_per_category', 'revenue':'mean_revenue_per_category',
                   'item_price':'mean_item_price_per_category'},
          inplace=True)
df_train_month = pd.merge(df_train_month, aux, on=['date_block_num', 'item_category_id'], how='left')
df_train_month.head()

,date_block_num,shop_id,item_id,item_cnt_month,revenue_month,mean_item_price_per_shop,mean_item_cnt_per_shop,mean_revenue_per_shop,mean_item_price_per_item,mean_item_cnt_per_item,mean_revenue_per_item,item_category_id,type_code,subtype_code,mean_item_price_per_category,mean_item_cnt_per_category,mean_revenue_per_category
0,0,0,32,6.0,1326.0,531.262049,1.163781,618.90507,338.110349,1.328889,451.181600,40,11,4,244.485942,1.145902,296.569262
1,0,0,33,3.0,1041.0,531.262049,1.163781,618.90507,488.517241,1.051724,514.327586,37,11,1,465.036007,1.053414,491.797659
2,0,0,35,1.0,247.0,531.262049,1.163781,618.90507,390.709091,1.418182,549.272727,40,11,4,244.485942,1.145902,296.569262
3,0,0,43,1.0,221.0,531.262049,1.163781,618.90507,234.500000,1.000000,234.500000,40,11,4,244.485942,1.145902,296.569262
4,0,0,51,2.0,257.0,531.262049,1.163781,618.90507,230.461538,1.000000,230.461538,57,13,8,277.780085,1.003914,278.672453


### Média dos itens por loja

#### Não sei calcular a média do número de itens vendidos por loja.

In [ ]:
col = ['date_block_num', 'item_price', 'item_cnt_day', 'revenue', 'item_id', 'shop_id']
group_index = ['date_block_num', 'shop_id', 'item_id']

In [ ]:
aux = train[col].groupby(group_index).mean()

aux.rename(columns={'item_cnt_day':'item_cnt-(mean_shop_item)',
                    'revenue':'revenue-(mean_shop_item)', 'item_price':'item_price-(mean_shop_item)'}, inplace=True)

#train = pd.merge(train, aux, left_on=group_index, right_index=True, how='inner')

aux.head()

In [ ]:
train[(train.shop_id == 0) & (train.date_block_num == 0) & (train.item_id == 32)]['item_cnt_day']

### Lagged Variables

Uma lagged feature é obtida ao atrasarmos ou adiantarmos a sequência daquela feature. Como por exemplo,

| Vendas_original  | Vendas_lag1    |
|----------|:------------- |
| 1 |  Nan |
| 2 |    1   |
| 3 | 2 |

A ideia por trás de fazermos essa transformação, é dar ao algoritmo a oportunidade de fazer escolhas como:
- O número de vendas desse mês é o mesmo número do mês passado.
- Dependendo do tamanho do lag é possível que o algoritmo aprenda sobre sazonalidade das vendas. Um bom exemplo é o natal, todo ano existe um aumento nas vendas neste período e o algoritmo pode levar isso em consideração.

<a name="lagged"></a> Deixei no apéndice uma explicação mais extensa sobre os [lagged variables](#lagged).

> Daqui pra frente as coisas ficam complicadas e eu não sei se vou utilizar lagged variables para treinar o modelo, mas já deixei o dataframe quase pronto para ser colocado em uma [6. LSTM](6.%20LSTM%20(Simples).ipynb).

In [39]:
df_train_month.head()

,date_block_num,shop_id,item_id,item_cnt_month,revenue_month,mean_item_price_per_shop,mean_item_cnt_per_shop,mean_revenue_per_shop,mean_item_price_per_item,mean_item_cnt_per_item,mean_revenue_per_item,item_category_id,type_code,subtype_code,mean_item_price_per_category,mean_item_cnt_per_category,mean_revenue_per_category
0,0,0,32,6.0,1326.0,531.262049,1.163781,618.90507,338.110349,1.328889,451.181600,40,11,4,244.485942,1.145902,296.569262
1,0,0,33,3.0,1041.0,531.262049,1.163781,618.90507,488.517241,1.051724,514.327586,37,11,1,465.036007,1.053414,491.797659
2,0,0,35,1.0,247.0,531.262049,1.163781,618.90507,390.709091,1.418182,549.272727,40,11,4,244.485942,1.145902,296.569262
3,0,0,43,1.0,221.0,531.262049,1.163781,618.90507,234.500000,1.000000,234.500000,40,11,4,244.485942,1.145902,296.569262
4,0,0,51,2.0,257.0,531.262049,1.163781,618.90507,230.461538,1.000000,230.461538,57,13,8,277.780085,1.003914,278.672453


# teste:

In [48]:
aux = df_train_month[['date_block_num', 'shop_id', 'item_id', 'item_cnt_month', 'revenue_month']].pivot_table(
                index=['shop_id', 'item_id'],
                columns=['date_block_num'],
                values=['item_cnt_month', 'revenue_month'], fill_value=0)
aux.head()

item_cnt_month                              ... revenue_month  \
date_block_num              0   1  2  3  4  5  6  7  8  9   ...            24   
shop_id item_id                                             ...                 
0       30                   0  31  0  0  0  0  0  0  0  0  ...           0.0   
        31                   0  11  0  0  0  0  0  0  0  0  ...           0.0   
        32                   6  10  0  0  0  0  0  0  0  0  ...           0.0   
        33                   3   3  0  0  0  0  0  0  0  0  ...           0.0   
        35                   1  14  0  0  0  0  0  0  0  0  ...           0.0   

                                                              
date_block_num    25   26   27   28   29   30   31   32   33  
shop_id item_id                                               
0       30       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
        31       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
        32       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
        33       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
        35       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 68 columns]

___________________

Adicionando as lagged em item_cnt_day e revenue.

In [ ]:
first_lvl_columns = df_month.columns.levels[0] # [item_cnt_day, revenue]
n_lags = 1

for first_lvl_name in first_lvl_columns:
    
    for lag in range(1,n_lags + 1,1):
        
        aux = df_month[first_lvl_name].shift(-lag, axis=1)
        
        aux = pd.concat([aux], keys=[first_lvl_name + '_lag(-'+str(lag)+')'], axis=1) # Faltou habilidade aqui.
        
        df_month = pd.concat([df_month, aux], join='inner', axis=1)

df_month.head()

In [ ]:
# Uma visualização mais limpa.
df_month[['item_cnt_day_lag(-1)']].head()

É necessário retirar as colunas com o mês 33. O motivo disso está um pouco explicado aqui [lagged variables](#lagged).

### Semestre e ano

A ideia aqui é criar colunas que indicam meses especiais com 1 e meses normais com 0. Por exemplo, podemos criar uma coluna na qual o mês de Dezembro é considerado especial (1). Isso deve ajudar o algoritmo a prever as vendas de natal.

In [ ]:
# Criando uma coluna com zeros.
train['is_year'] = 0

# Nos lugares onde o date_block_num for divisível por 12 colocamos o valor 1.
train['is_year'][(train.date_block_num + 1) % 12 == 0]  = 1

> <a name="back3"></a> **Obs.:** Se ficou muito complicado de entender o que aconteceu na última linha, eu coloquei um exemplo mais simples [aqui](#mask).

In [ ]:
# Checando se os valores foram substituidos corretamente.
train[train.date_block_num == 11].head()

In [ ]:
# Criando uma coluna com zeros.
train['is_semester'] = 0

# Nos lugares onde o date_block_num for divisível por 12 colocamos o valor 1.
train['is_semester'][(train.date_block_num + 1) % 6 == 0]  = 1

In [ ]:
# Checando se os valores foram substituidos corretamente.
train[train.date_block_num == 11].head()

In [ ]:
# Checando se os valores foram substituidos corretamente.
train[train.date_block_num == 5].head()

### Latitude e Longitude

No final do notebook [1. EDA](1.%20EDA.ipynb) (Sessão Show off) eu queria colocar alguns mapas com as localizações e vendas das lojas, mas acabei achando interessante incluir também a latitude e longitude como features.
Para criar o arquivo com as latitudes e longitudes eu usei a feature com o nome das cidades e esse [site](https://www.latlong.net/).

### <a name="unindo"></a> Unindo os dataframes (sem lagged)

In [ ]:
df_clean = pd.merge(train, shops, on=['shop_id'], how='left')
df_clean = pd.merge(df_clean, items, on=['item_id'], how='left')
df_clean = pd.merge(df_clean, items_cat, on=['item_category_id'], how='left')

df_clean.drop(['shop_name', 'item_name', 'city',
         'item_category_name', 'split',
         'type', 'subtype'], axis=1, inplace=True)

-----------------

# Apêndice

### Trocando features categóricas por números ou vetores

<a name="go1"></a>No código abaixo, x vai guardar a relações entre os rótulos,
-  Ex.: a = 1, b = 2, e etc.

Isso é importante se quisermos fazer a transformação reversa.

[Voltar ao notebook](#back1)

In [ ]:
x = LabelEncoder()

vetor_normal = ['a','b','c','d']

vetor_transformado = x.fit_transform(vetor_normal)

print('Categorias', vetor_normal)
print('Rótulos:', vetor_transformado)

voltando = x.inverse_transform(vetor_transformado)

print('Invertendo a transformação:', voltando)

One-hot enconding.

In [ ]:
one = OneHotEncoder(handle_unknown='ignore')
vetor = [['Laranja', 'bom'], ['Maçã', 'bom'], ['Cebola', 'ruim']]
one.fit(vetor)

In [ ]:
one_hot = one.transform(vetor).toarray()
print('Representação em forma de vetor:')
print(['Laranja', 'bom'], '=', one_hot[0])
print(['Maçã', 'bom'], '=', one_hot[1])
print(['Cebola', 'ruim'], '=', one_hot[2])

Quando colocamos um vetor que não existe, como ['Chocolate, 'médio'], e o handle_unknown='ignore', a função vai retornar um vetor $0$. Agora, se colocarmos um vetor que não estava presente inicialmente, mas é uma combinação dos outros como ['Cebola', 'bom'], então conseguimos uma representação vetorial.

In [ ]:
one_hot = one.transform([['Chocolate', 'médio'], ['Cebola', 'bom']]).toarray()
print('Não estava presente nas categórias:')
print(['Chocolate', 'médio'], '=', one_hot[0])
print('Combinação de outras categórias:')
print(['Cebola', 'bom'], '=', one_hot[1])

----------------------------------------

### <a name="lagged"></a>  Lagged um pouco mais explicada

[Voltar ao notebook](#back2)

In [ ]:
a = pd.merge(a, a.shift(-1, axis=1), left_index=True, right_index=True, how='inner', suffixes=('_-1', '_0'))

Explicando lagged variables

In [ ]:
# Criando um pandas dataframe com o número de vendas de 1 até 6.
df_aux = pd.DataFrame(np.linspace(1,6,6), columns=['Vendas'])
df_aux

> **Obs.:** df_aux simula um dataset das vendas de uma loja, onde a passagem de tempo seria algo do tipo:
- linha 0 = mês de Janeiro.
- linha 1 = mês de Fevereiro.

In [ ]:
#Usando a função shift para deslocar o array em 1.
df_aux['Vendas_shift_+'] = df_aux['Vendas'].shift(1)
df_aux['Vendas_shift_-'] = df_aux['Vendas'].shift(-1)

df_aux

Temos que descartar as linhas que possuem Nan's.

In [ ]:
df_aux.dropna(inplace=True)
df_aux

Note que as colunas estão fora de ordem, para que o algoritmo reconheça a passagem de tempo é necessário reorganziar as colunas.

In [ ]:
df_aux = df_aux[['Vendas_shift_+', 'Vendas', 'Vendas_shift_-']]
df_aux

É mais conveninente renomear as colunas tmb.

In [ ]:
df_aux.rename(columns={'Vendas_shift_+': 'Vendas_t-2',
                       'Vendas': 'Vendas_t-1',
                       'Vendas_shift_-': 'Vendas_t'}
              , inplace=True)
df_aux

Compare essa tabela com a gerada inicialmente, onde a passagem dos meses seguiam o número das linhas. Na nova tabela, a passagem de tempo pode ser vista seguindo as colunas $t-2$, $t-1$, $t$.

> **Obs.:** Esse tipo de organização é perfeita para usarmos em algoritmo de Machine Learning. As linhas seriam suas amostras, as features seriam $t-2$ e $t-1$, e o que você estaria prevendo seria $t$.

### Código para gerar e organizar as variáveis com lag

O processo de criar lagged features é bem chato se for feito manualmente igual fizemos acima, principalmente se fizermos com muitas variáveis. Acabei fazendo um função pra fazer isso. Ela foi baseado em outra função encontrada [aqui][lag] (ótimo site para aprender Machine Learning).

[lag]: https://machinelearningmastery.com/basic-feature-engineering-time-series-data-python/

> Acabei não usando essa função nesse notebook, mas ela é bem útil para gerar lagged variables.

In [ ]:
def time_shift(df, lags, dropnan=True):
    
    '''
    Adiciona novas colunas deslocadas no tempo.
    Argumentos:
    
        df: Pandas DataFrame.
        
        lags: lista com o número de passos
               a ser deslocado.
               
        dropnan: (True) Retira as linhas que
                teham Nan's.
                
    Retorna:
        Pandas DataFrame com as colunas deslocadas.
    '''
    lags.append(0)
    lags = -1 * np.sort(lags)
    
    columns = [str(df.name)] if type(df) is pd.Series else df.columns
    col = []
    
    for column in columns:
        for lag in lags: 
            
            if lag == 0:
                new_column = column 
                col.append(new_column)
            else:
                new_column = column + '_lag(' + str(-1 * lag) + ')'
                col.append(new_column)
                df[new_column] = df[column].shift(lag)
                
    if dropnan:
        df.dropna(inplace=True)
        
    return df[col]

### <a name="mask"></a> Exemplo para seleção do mês ano

[Voltar ao notebook](#back3)

Vamos criar um array com números de 1 a 36.

In [ ]:
month = np.linspace(1,36,36)
month

Ao fazermos (month % 12), o python retorna o resto da divisão do array month por 12, ou seja, os únicos números que divisíveis são 12, 24, 36.

In [ ]:
(month % 12)

Agora, ao adicionarmos uma comparação (month % 12 == 0), queremos saber em quais posições do arrya a condição (== 0) é verdade.

In [ ]:
(month % 12 == 0)

Logo, ao colocarmos esse tipo de array no pandas queremos dizer: Só faça o que pedimos nas posições onde temos **True**, não faça nada se for **False**.

----------------------------------------

# Referências
1. Muitas das features craidas neste notebook foram retiradas deste [kernel](https://www.kaggle.com/dlarionov/feature-engineering-xgboost).
2. Site de onde tirei as informações sobre lagged variables. [Machine Learning Mastery](https://machinelearningmastery.com/basic-feature-engineering-time-series-data-python/)
3. Site para encontrar as latitudes e longitudes. [Latlong](https://www.latlong.net/).